### Helper Utils
2,5,6

In [220]:
def generateFragments(peptide):
    length = len(peptide)
    cycle = [ peptide + peptide[:i] for i in range(length) ]
    fragments = [cycle[i][k:k+i+1] for i in range(length-1) for k in range(length)]
    fragments.append(peptide)

    return fragments
def generateSpectrum(peptide, mass_table = masses):
    fragments = generateFragments(peptide)
    spectrum = []
    spectrum.append(0)
    for f in fragments :
        mass = 0
        for i,p in enumerate(f):
            mass += mass_table [p]
        spectrum.append(mass)
    
    return spectrum

mass_peptide = {
        'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,\
        'I':113,'N':114,'D':115,'E':129,'K':128, 'M':131,'H':137,\
        'F':147,'R': 156,'Y': 163,'W': 186
        }   

masses =  {'G': 57, 'A': 71, 'S': 87, 'P': 97, 'V': 99, 'T': 101, 'C': 103, 'I': 113, 'L': 113, 'N': 114, 
                'D': 115, 'K': 128, 'Q': 128, 'E': 129, 'M':131, 'H': 137, 'F': 147, 'R': 156, 'Y': 163, 'W': 186}    

### Compute the Score of a Cyclic Peptide Against a Spectrum

In [186]:
def scoreCyclopeptide(peptide, exp_spectrum, mass_table = masses):
    # print("peptide: ", peptide)
    # print("exp_spectrum: ",exp_spectrum)
    theo_spectrum = generateSpectrum(peptide, mass_table)
    # print("theo_spectrum: ",theo_spectrum)
    lst3 = [value for value in theo_spectrum if value in exp_spectrum] 
    return len(lst3)

In [22]:
f= open("test.txt","r")
lines = f.readlines()
peptide_ = lines[0].strip()
ex_spectrum = lines[1].strip().split()
ex_spectrum = [int(i) for i in ex_spectrum] 
scoreCyclopeptide(peptide_, ex_spectrum)

peptide:  NQEL
exp_spectrum:  [0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484]
theo_spectrum:  [0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]


11

### Implement LeaderboardCyclopeptideSequencing

In [207]:
def Expand(board, mass_table = masses):
    new_peptides = []
    for peptide in board:
        for acid in mass_table:
            new_peptides.append(peptide + acid)
    return new_peptides


def Mass(Peptide, mass_table = masses):
    if Peptide in cyclospectra:
        return cyclospectra[Peptide]
    else:
        spectrum = [0, int(sum(mass_table[aa] for aa in Peptide))]    
        cycle = Peptide * 2
        for i in range( len( Peptide )):
            for j in range( i + 1, i + len( Peptide )):
                spectrum.append( sum( mass_table[aa] for aa in cycle[i:j] ))            

        cyclospectra[Peptide] = spectrum    
        return cyclospectra[Peptide]
    
cyclospectra = {}

def leaderboardCyclopeptideSequencing(Spectrum, N, mass_table = masses):

    Parent_Mass = Spectrum[-1]     
    
    Leaderboard = ['']
    LeaderPeptide = ''
    
    while Leaderboard:

        Leaderboard = Expand(Leaderboard, mass_table)
        scores = []        

        for Peptide in Leaderboard:

            score = scoreCyclopeptide(Peptide, Spectrum,mass_table)   
            Mass_Peptide = max(Mass(Peptide, mass_table))
            if Mass_Peptide == Parent_Mass and score > scoreCyclopeptide(LeaderPeptide, Spectrum,mass_table):
                LeaderPeptide = Peptide

            elif Mass_Peptide > Parent_Mass:
                score = 0

            scores.append(score)  

        if len(Leaderboard) > N:

            cut_off = sorted(scores)[-N]
            leaders = []

            for i in range(len(Leaderboard)):
                if scores[i] >= cut_off and scores[i] > 0:
                    leaders.append(Leaderboard[i])

            Leaderboard = leaders 
    
    return LeaderPeptide




In [208]:
f = open("test.txt",'r')

N = int(f.readline())
Spectrum = [int(i) for i in f.readline().split(' ')]

Leader = leaderboardCyclopeptideSequencing(Spectrum, N)
masses = [str(mass[aa]) for aa in Leader]
string = '-'.join(masses)
print(string)

71-147-113-129


### Generate the Convolution of a Spectrum

In [209]:
import operator
def generateSpectralConvolution(Spectrum):
    convolution_array = []
    for i in Spectrum :
        for j in Spectrum:
            if i - j > 0:
                convolution_array.append(i-j)
    

    counts = Counter([i for i in convolution_array])
    result = []
    for k in sorted(counts, key=lambda x: counts[x], reverse=True):
            result += [k] * counts[k]
    # print("generate Spectral convolution: ", result)
    tables = dict(counts)

    return result,tables

In [156]:
from collections import Counter
 
f = open("test.txt",'r')
Spectrum = [int(i) for i in f.readline().split(' ')]
convolution, _ = generateSpectralConvolution(Spectrum)


generate Spectral convolution:  [137, 137, 186, 186, 49, 323]


{137: 2, 186: 2, 49: 1, 323: 1}

### ConvolutionCyclopeptideSequencing

In [222]:
import copy
def trim_spectrum(table,M):
    newtable = {}
    for k in table.keys():
        if k >= 57 and k <= 200:
            newtable[k] = table[k]
    allValues = sorted(newtable.values(), reverse = True)
    if M > len(allValues):
        val_to_keep = allValues
    else:
        thres = allValues[M]
        val_to_keep = [k for k in newtable.keys() if newtable[k] >= thres]
    return val_to_keep

def ConvolutionCyclopeptideSequencing(M, N, Spectrum, mass_table =  masses) :
    conv,table = generateSpectralConvolution(Spectrum)
    trim = trim_spectrum(table,M)
    # print("Trimmed_spectrum: ", trim)
    simple_table = copy.copy(mass_table)
    for k,v in mass_table.items():
        if v not in trim:
            del simple_table[k]

    placeholders = 'abcdefghijklmnopqrstuvwxyz'
    start = 0
    for v in trim:
        if v not in simple_table.values():
            simple_table[placeholders[start]] = v
            start += 1
    print(simple_table)
    leader = leaderboardCyclopeptideSequencing(Spectrum, N, simple_table)
    print("leader: ", leader)
    masses = [str(simple_table[aa]) for aa in leader]
    string = '-'.join(masses)
    print(string)


In [224]:
with open('test.txt', 'r') as f: 
    M_ = int(f.readline())
    N_ = int(f.readline())
    Spectrum_ = [int(i) for i in f.readline().split(' ')]
    print("M: ", M_,"\nN: ", N_,"\nSpectrum: ", Spectrum_)
ConvolutionCyclopeptideSequencing(M_,N_, Spectrum_)

M:  20 
N:  60 
Spectrum:  [57, 57, 71, 99, 129, 137, 170, 186, 194, 208, 228, 265, 285, 299, 307, 323, 356, 364, 394, 422, 493]
{'G': 57, 'A': 71, 'S': 87, 'V': 99, 'I': 113, 'L': 113, 'D': 115, 'K': 128, 'Q': 128, 'E': 129, 'H': 137, 'R': 156, 'W': 186, 'a': 72, 'b': 58, 'c': 80, 'd': 66, 'e': 123, 'f': 95, 'g': 65, 'h': 151, 'i': 109, 'j': 79, 'k': 171, 'l': 157, 'm': 91, 'n': 194, 'o': 166, 'p': 136, 'q': 148, 'r': 200, 's': 170, 't': 162, 'u': 178}
leader:  GEVAbj
57-129-99-71-58-79


### Construct a Trie from a Collection of Patterns

In [230]:
def trieConstruction(Patterns):
    trie = {0:{}}
    idx = 1
    for pattern in patterns:
        curr = 0
        for base in pattern:
            if base not in trie[curr]:
                trie[curr][base] = idx
                trie[idx] = {}
                curr = idx 
                idx += 1
            else:
                curr = trie[curr][base]
    return trie

def printTrie(trie):
    for node in trie:
        for symbol in trie[node]:
            print(str(node)+ '->'+str(trie[node][symbol])+':'+symbol)



In [232]:
with open('test.txt') as f:
    patterns = [pattern.strip() for pattern in f.readlines()]

trie = trieConstruction(patterns)
printTrie(trie)

0->1:A
0->7:G
1->2:T
2->3:A
2->6:C
3->4:G
4->5:A
7->8:A
8->9:T


### Implement TrieMatching

In [234]:
def prefixMatching(text, trie):
    char = text[0]
    node = 0
    path_length = 0
    path = ""
    while True:
        if len(trie[node]) == 0 :
            return path
        elif char in trie[node]:
            path = path + char
            node = trie[node][char]
            path_length += 1
            if path_length < len(text) :
                char = text [path_length]
            else:
                pass
        else:
            return

def trieMatching( patterns, text):
    trie = trieConstruction(patterns)
    results = []
    for i in range(len(text) - 1):
        results.append(prefixMatching(text[i:], trie))
    return results 

In [239]:
f = open('test.txt', "r")
text = f.readline().strip()
patterns = [line.strip() for line in f.readlines()]

result = trieMatching(patterns, text)
print (' '.join([str(i) for i in range(len(result)) if result[i] != None]))


1 4 11 15


### Construct the Suffix Array of a String

In [244]:
def suffixArrayConstruction(text):
    ordered_pairs = sorted([(text[i:],i) for i in range(len(text))])
    print(ordered_pairs)
    suffix_array = [pair[1] for pair in ordered_pairs]
    return suffix_array



In [245]:
f = open('test.txt', "r")
text = f.readline().strip()
suffixArrayConstruction(text)

[('$', 15), ('A$', 14), ('AACGATAGCGGTAGA$', 0), ('ACGATAGCGGTAGA$', 1), ('AGA$', 12), ('AGCGGTAGA$', 6), ('ATAGCGGTAGA$', 4), ('CGATAGCGGTAGA$', 2), ('CGGTAGA$', 8), ('GA$', 13), ('GATAGCGGTAGA$', 3), ('GCGGTAGA$', 7), ('GGTAGA$', 9), ('GTAGA$', 10), ('TAGA$', 11), ('TAGCGGTAGA$', 5)]


[15, 14, 0, 1, 12, 6, 4, 2, 8, 13, 3, 7, 9, 10, 11, 5]